In [ ]:
!pip install spatial

  Preparing metadata (setup.py) ... done
  Created wheel for spatial: filename=spatial-0.2.0-py3-none-any.whl size=8484 sha256=3c4a82641f38540c9b2e3a5fd8c37b71560ea0f14ce0a0e92271ba484a8b995e
  Stored in directory: /root/.cache/pip/wheels/b3/a9/45/5776d9357e179ab80f60dfbf7d002d095f1a7c5d8f86104bd3
Successfully built spatial


In [ ]:
!pip install scipy

In [ ]:
!pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install dtaidistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.6 MB/s eta 0:00:00


In [ ]:
# The idea of this notebook is to use the pretrained model VGG19 and measure similarity

In [ ]:
# Import the necessary packages, modules, functions, and methods
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import torch
import torchvision
import cv2
import spatial
import scipy.spatial.distance
from keras.layers import Flatten, Dense, Input,concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout
from keras.models import Model
from keras.models import Sequential
import tensorflow as tf
import sktime
from sktime.distances import euclidean_distance
from dtaidistance import dtw
import dtaidistance
from sklearn import preprocessing

In [ ]:
# mount my google drive so I can import files from there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# we load the VGG19 pre-trained model
vgg19 = keras.applications.VGG19(weights='imagenet', include_top=True, pooling='max', input_shape=(224, 224, 3))

574710816/574710816 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
#show the layers of VGG19
vgg19.summary()

Model: "vgg19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 143,667,240 (548.05 MB)

 Trainable params: 143,667,240 (548.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# not all layers of the model are used. We extract the vector from layer “fc2” as an extracted feature of an image
model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('fc2').output)

In [ ]:
#below we define three functions that we use to calcualte similarity

In [ ]:
# function retrieves the feature vector of an image
def get_feature_vector(img):
  # Convert PyTorch tensor to NumPy array
  img_np = img.permute(1, 2, 0).numpy() # Change the order of dimensions
  img1 = cv2.resize(img_np, (224, 224))
  # Convert NumPy array back to PyTorch tensor
  img1_tensor = torch.from_numpy(img1).permute(2, 0, 1) # Change the order of dimensions
  feature_vector = model.predict(img1_tensor.reshape(1, 224, 224, 3))
  return feature_vector.flatten() #copy of the array collapsed into one dimension

In [ ]:
# function for calculating cosine similarity of vectors
def cosine_similarity(vector1, vector2):
  return 1 - scipy.spatial.distance.cosine(vector1, vector2) # Use scipy.spatial.distance to caluate the cosine similarity of the vectors

In [ ]:
# function for calculating euclidean similarity of vectors
def euclidean_similarity(x, y):
    return 1 / (1 + euclidean_distance(x, y)) # Use sktime.distances to caluate the euclidean similarity of the vectors

In [ ]:
# function for calculating dynamic time warping similarity of vectors
def dtw_similarity(time_series_a,time_series_b):
  # dtw.warping_paths returns the DTW distance and the DTW cost matrix
  distance, paths = dtw.warping_paths(time_series_a.flatten(), time_series_b.flatten(), use_c=False)
  # Compute the optimal path from the DTW cost matrix
  best_path = dtw.best_path(paths)
  return 1 - (distance / len(best_path)) # Use dtaidistance to caluate the dynamic time warping similarity of the vectors

# function for scaling data in an array
def normalize_series(data):
  # collapse the array into one dimension
  data_array = data.flatten()
  # Normalize the data using MinMaxScaler
  scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
  # Fit the scaler on the data and transform it
  normalized_array = scaler.fit_transform(data_array.reshape(-1, 1))
  return normalized_array # return the normalized array

In [ ]:
#We check the similarity between similar songs involved in famous court cases using cosine similarity, euclidean similarity, and dynamic time warping similarity
#In each case the closer the score is to 0 the more dissimilar the songs are while the closer the score is to 1 the more similiar the songs are

In [ ]:
# Vanilla Ice's Ice Ice Baby vs. Under Pressure by Queen and David Bowie
copy1 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IceIceBaby_full.jpg")
org1 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/UnderPressure_full.jpg")
f1 = get_feature_vector(org1)
f2 = get_feature_vector(copy1)
cosine_similarity(f1, f2), euclidean_similarity(f1, f2), dtw_similarity(normalize_series(f1),normalize_series(f2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step


(0.8055741270090152, 0.02011798958772857, 0.9994012023604543)

In [ ]:
# the Verve's Bitter Sweet Symphony vs. the Rolling Stone's The Last Time
org2 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/BittersweetSymphony_full.jpg")
copy2 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheLastTime_full.jpg")
v1 = get_feature_vector(org2)
v2 = get_feature_vector(copy2)
cosine_similarity(v1, v2), euclidean_similarity(v1, v2), dtw_similarity(normalize_series(v1),normalize_series(v2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 728ms/step


(0.8790536565376178, 0.027354739803946378, 0.9994327812462329)

In [ ]:
# Robin Thicke featuring rapper T.I. and singer Pharrell Williams's Blurred Lines vs. Marvin Gaye's Gotta Give It Up
org3 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/GottaGiveItUp_full.jpg")
copy3 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/blurredlines_full.jpg")
a1 = get_feature_vector(org3)
a2 = get_feature_vector(copy3)
cosine_similarity(a1, a2), euclidean_similarity(a1, a2), dtw_similarity(normalize_series(a1),normalize_series(a2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 826ms/step


(0.8771590988367582, 0.027306916917929934, 0.9994705188166622)

In [ ]:
# Killing Joke‘s Eighties vs. Nirvana's Come as You are
copy4 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ComeAsYouAre_full.jpg")
org4 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheKillingJoke_full.jpg")
b1 = get_feature_vector(org4)
b2 = get_feature_vector(copy4)
cosine_similarity(b1, b2), euclidean_similarity(b1, b2), dtw_similarity(normalize_series(b1),normalize_series(b2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 716ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.8083039887032799, 0.019844630618943343, 0.9995117160289467)

In [ ]:
# Chuck Berry's You Can't Catch Me vs. The Beatles Come Together
org5 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouCantCatchMe_full.jpg")
copy5 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ComeTogether_full.jpg")
c1 = get_feature_vector(org5)
c2 = get_feature_vector(copy5)
cosine_similarity(c1, c2), euclidean_similarity(c1, c2), dtw_similarity(normalize_series(c1),normalize_series(c2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 946ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.7864741710988497, 0.017530893036502043, 0.9993226566550568)

In [ ]:
# Radiohead's Creep vs. Lana Del Rey's Get Free
org6 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Creep_full.jpg")
copy6 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/GetFree_full.jpg")
d1 = get_feature_vector(org6)
d2 = get_feature_vector(copy6)
cosine_similarity(d1, d2), euclidean_similarity(d1, d2), dtw_similarity(normalize_series(d1),normalize_series(d2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.6797330615115218, 0.016030223128876482, 0.9993604400906381)

In [ ]:
# Radiohead's Creep vs. The Hollies The Air That I Breathe
org7 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheAirThatIBreathe_full.jpg")
copy7 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Creep_full.jpg")
e1 = get_feature_vector(org7)
e2 = get_feature_vector(copy7)
cosine_similarity(e1, e2), euclidean_similarity(e1, e2), dtw_similarity(normalize_series(e1),normalize_series(e2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step


(0.8291483446927449, 0.020238894366345572, 0.999343952859248)

In [ ]:
# George Harrison's My Sweet Lord vs. the Chiffons’ He’s So Fine
org8 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Hessofine_full.jpg")
copy8 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/MySweetLord_full.jpg")
g1 = get_feature_vector(org8)
g2 = get_feature_vector(copy8)
cosine_similarity(g1, g2), euclidean_similarity(g1, g2), dtw_similarity(normalize_series(g1),normalize_series(g2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 724ms/step


(0.7291817688629534, 0.017234220027900985, 0.999382497903564)

In [ ]:
# Katy Perry featuring Juicy J.'s Dark Horse vs. Flame's Joyful Noise
org9 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/JoyfulNoise_full.jpg")
copy9 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/DarkHorse_full.jpg")
h1 = get_feature_vector(org9)
h2 = get_feature_vector(copy9)
cosine_similarity(h1, h2), euclidean_similarity(h1, h2), dtw_similarity(normalize_series(h1),normalize_series(h2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 739ms/step


(0.8519644914038389, 0.02204289832425731, 0.9994164809513911)

In [ ]:
# Rod Stewart's Do You Think I'm Sexy vs. Jorge Ben jor's Taj Mahal
org10 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TajMahal_full.jpg")
copy10 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/DoYaThinkImSexy_full.jpg")
k1 = get_feature_vector(org10)
k2 = get_feature_vector(copy10)
cosine_similarity(k1, k2), euclidean_similarity(k1, k2), dtw_similarity(normalize_series(k1),normalize_series(k2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.8291427724288306, 0.021096488392500123, 0.9994054504767954)

In [ ]:
# Ray Parker Jr.'s Ghostbusters vs. Huey Lewis's I Want a New Drug
copy11 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Ghostbusters_full.jpg")
org11 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IWantaNewDrug_full.jpg")
m1 = get_feature_vector(org11)
m2 = get_feature_vector(copy11)
cosine_similarity(m1, m2), euclidean_similarity(m1, m2), dtw_similarity(normalize_series(m1),normalize_series(m2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 740ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step


(0.8718453174738019, 0.022658535458720246, 0.9993114188219232)

In [ ]:
# Olivia Rodrigo's Good 4 U vs. Paramore's Misery Business
org12 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/MiseryBusiness_full.jpg")
copy12 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Good4u_full.jpg")
n1 = get_feature_vector(org12)
n2 = get_feature_vector(copy12)
cosine_similarity(n1, n2), euclidean_similarity(n1, n2), dtw_similarity(normalize_series(n1),normalize_series(n2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.7440031348574517, 0.014813687813969753, 0.999345251184862)

In [ ]:
# Oasis's Shakermaker vs. The New Seekers's I’d Like To Teach The World To Sing
org13 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/I'dLiketoTeaktheworldtosing_full.jpg")
copy13 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Shakermaker_full.jpg")
p1 = get_feature_vector(org13)
p2 = get_feature_vector(copy13)
cosine_similarity(p1, p2), euclidean_similarity(p1, p2), dtw_similarity(normalize_series(p1),normalize_series(p2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 722ms/step


(0.6076017590963336, 0.014830734584062578, 0.9993896590609864)

In [ ]:
# Marvin Gaye's Let's Get It On vs. Ed Sheeran's Thinking Out Loud
org14 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/LetsGetItOn_full.jpg")
copy14 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ThinkingOutLoud_full.jpg")
y1 = get_feature_vector(org14)
y2 = get_feature_vector(copy14)
cosine_similarity(y1, y2), euclidean_similarity(y1, y2), dtw_similarity(normalize_series(y1),normalize_series(y2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step


(0.9326566752837794, 0.031189998703081926, 0.9994284840609174)

In [ ]:
# Led Zeppelin's Stairway to Heaven vs. Spirit's Taurus
org15 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Taurus_full.jpg")
copy15 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/StairwaytoHeaven_full.jpg")
q1 = get_feature_vector(org15)
q2 = get_feature_vector(copy15)
cosine_similarity(q1, q2), euclidean_similarity(q1, q2), dtw_similarity(normalize_series(q1),normalize_series(q2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 730ms/step


(0.7709411082810511, 0.017568297871666392, 0.9993847252310363)

In [ ]:
# Sam Smith's Stay with Me vs. Tom Petty's I won't Back Down
org16 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IWontBackDown_full.jpg")
copy16 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/StayWithMe_full.jpg")
r1 = get_feature_vector(org16)
r2 = get_feature_vector(copy16)
cosine_similarity(r1, r2), euclidean_similarity(r1, r2), dtw_similarity(normalize_series(r1),normalize_series(r2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 860ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 749ms/step


(0.9050632810703388, 0.02595942363659112, 0.9994314638283996)

In [ ]:
# Beach Boy's Surfin USA vs. Chuck Berry's Sweet Little Sixteen
org17 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/SweetLittleSixteen_full.jpg")
copy17 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/SurfinUSA_full.jpg")
s1 = get_feature_vector(org17)
s2 = get_feature_vector(copy17)
cosine_similarity(s1, s2), euclidean_similarity(s1, s2), dtw_similarity(normalize_series(s1),normalize_series(s2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 746ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step


(0.8890189897757982, 0.023326395971054938, 0.9993773887040958)

In [ ]:
# De La Soul's Transmitting Live From Mars vs. The Turtles's You Showed Me
org18 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouShowedMe_full.jpg")
copy18 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TransmittingLivefromMars_full.jpg")
t1 = get_feature_vector(org18)
t2 = get_feature_vector(copy18)
cosine_similarity(t1, t2), euclidean_similarity(t1, t2), dtw_similarity(normalize_series(t1),normalize_series(t2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 905ms/step


(0.6688337191870799, 0.015064543625531378, 0.9993237850851527)

In [ ]:
# Coldplay's Viva La Vida vs. Joe Satriani's If I Could Fly
org19 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IfICouldFly_full.jpg")
copy19 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/VivaLaVida_full.jpg")
w1 = get_feature_vector(org19)
w2 = get_feature_vector(copy19)
cosine_similarity(w1, w2), euclidean_similarity(w1, w2), dtw_similarity(normalize_series(w1),normalize_series(w2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.7104054889425818, 0.015664752364682094, 0.9994197449168208)

In [ ]:
# Led Zeppelin's Whole Lotta Love vs. Muddy Waters's You Need Love
org20 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouNeedLove_full.jpg")
copy20 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/WholeLottaLove_full.jpg")
x1 = get_feature_vector(org20)
x2 = get_feature_vector(copy20)
cosine_similarity(x1, x2), euclidean_similarity(x1, x2), dtw_similarity(normalize_series(x1),normalize_series(x2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 730ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step


(0.6717312053269706, 0.014947040612464213, 0.9993305129276191)

In [ ]:
#We check the similarity between songs that are very dissimilar to the human ear

In [ ]:
# J.S. Bach - Toccata and Fugue in D minor BWV 565 vs. Master Of Puppets - Metallica
org21 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/J.S. Bach - Toccata and Fugue in D minor BWV 565.jpg")
copy21 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Master Of Puppets - Metallica.jpg")
aa1 = get_feature_vector(org21)
aa2 = get_feature_vector(copy21)
cosine_similarity(aa1, aa2), euclidean_similarity(aa1, aa2), dtw_similarity(normalize_series(aa1),normalize_series(aa2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 734ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step


(0.6734261235560105, 0.014466070310431787, 0.9993463992746431)

In [ ]:
# Michael Jackson - Billie Jean vs. Alice Cooper - Schools Out
org22 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Michael Jackson - Billie Jean.jpg")
copy22 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Alice Cooper - Schools Out.jpg")
bb1 = get_feature_vector(org22)
bb2 = get_feature_vector(copy22)
cosine_similarity(bb1, bb2), euclidean_similarity(bb1, bb2), dtw_similarity(normalize_series(bb1),normalize_series(bb2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 756ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step


(0.7649259378752525, 0.017552581043070933, 0.9993341382458557)

In [ ]:
# Yakety Sax - Boots Randolph vs. Vanessa Carlton - A Thousand miles
org23 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Yakety Sax - Boots Randolph.jpg")
copy23 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Vanessa Carlton - A Thousand Miles.jpg")
cc1 = get_feature_vector(org23)
cc2 = get_feature_vector(copy23)
cosine_similarity(cc1, cc2), euclidean_similarity(cc1, cc2), dtw_similarity(normalize_series(cc1),normalize_series(cc2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 737ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 743ms/step


(0.8298599599291872, 0.01727608670340144, 0.9993880298668949)

In [ ]:
# AC⧸DC - Back In Black vs. 4'33'' - John Cage
org24 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/AC⧸DC - Back In Black.jpg")
copy24 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/4'33'' - John Cage.jpg")
dd1 = get_feature_vector(org24)
dd2 = get_feature_vector(copy24)
cosine_similarity(dd1, dd2), euclidean_similarity(dd1, dd2), dtw_similarity(normalize_series(dd1),normalize_series(dd2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 752ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step


(0.49915863902963886, 0.012105405397043448, 0.9994095551763876)

In [ ]:
# Tequila - The Champs  vs. Chopin - Nocturne in E Flat Major (Op. 9 No. 2)
org25 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Tequila - The Champs.jpg")
copy25 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Chopin - Nocturne in E Flat Major (Op. 9 No. 2).jpg")
ee1 = get_feature_vector(org25)
ee2 = get_feature_vector(copy25)
cosine_similarity(ee1, ee2), euclidean_similarity(ee1, ee2), dtw_similarity(normalize_series(ee1),normalize_series(ee2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 731ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 733ms/step


(0.4345144970456112, 0.012183433398181077, 0.9993376971935369)

In [ ]:
# Aretha Franklin - Respect vs. Love Theme From Romeo & Juliet - Pyotr Ilyich Tchaikovsky
org26 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Aretha Franklin - Respect.jpg")
copy26 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Love Theme From Romeo & Juliet - Pyotr Ilyich Tchaikovsky.jpg")
ff1 = get_feature_vector(org26)
ff2 = get_feature_vector(copy26)
cosine_similarity(ff1, ff2), euclidean_similarity(ff1, ff2), dtw_similarity(normalize_series(ff1),normalize_series(ff2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 731ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step


(0.6676220372723156, 0.015442123354573227, 0.9992892873749863)

In [ ]:
# Nirvana - Smells Like Teen Spirit vs. Ave Maria - J.S. Bach and Charles Gounod
org27 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Nirvana - Smells Like Teen Spirit.jpg")
copy27 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Ave Maria - J.S. Bach and Charles Gounod.jpg")
gg1 = get_feature_vector(org27)
gg2 = get_feature_vector(copy27)
cosine_similarity(gg1, gg2), euclidean_similarity(gg1, gg2), dtw_similarity(normalize_series(gg1),normalize_series(gg2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 757ms/step


(0.6179074669064047, 0.013960109367922737, 0.9992845687539661)

In [ ]:
# David Bowie - Heroes vs. OutKast - Hey Ya
org28 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/David Bowie - Heroes.jpg")
copy28 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/OutKast - Hey Ya.jpg")
hh1 = get_feature_vector(org28)
hh2 = get_feature_vector(copy28)
cosine_similarity(hh1, hh2), euclidean_similarity(hh1, hh2), dtw_similarity(normalize_series(hh1),normalize_series(hh2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 747ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step


(0.7577674840337784, 0.018379331292761317, 0.9993433709652925)

In [ ]:
# The Dave Brubeck Quartet - Take Five vs. Public Enemy - Fight the Power
org29 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/The Dave Brubeck Quartet - Take Five.jpg")
copy29 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Public Enemy - Fight the Power.jpg")
jj1 = get_feature_vector(org29)
jj2 = get_feature_vector(copy29)
cosine_similarity(jj1, jj2), euclidean_similarity(jj1, jj2), dtw_similarity(normalize_series(jj1),normalize_series(jj2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 755ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 749ms/step


(0.7245215260658412, 0.016515122473435655, 0.9993702292833168)

In [ ]:
# Grieg - Peer Gynt Suite No. 1, Op. 46 I. Morning Mood vs. Pour Some Sugar On Me - Def Leppard
org30 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Grieg - Peer Gynt Suite No. 1, Op. 46 I. Morning Mood.jpg")
copy30 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Pour Some Sugar On Me - Def Leppard.jpg")
kk1 = get_feature_vector(org30)
kk2 = get_feature_vector(copy30)
cosine_similarity(kk1, kk2), euclidean_similarity(kk1, kk2), dtw_similarity(normalize_series(kk1),normalize_series(kk2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


(0.66355388906099, 0.016361908415200174, 0.999244089441643)

In [ ]:
#The scores are all extremely high (close to 1) for all pairs of songs with respect to DTW similarity.
#The scores are all extremely low (close to 0) for all pairs of songs with respect to euclidean similarity.
#With respect to cosine similarity, for all the similar songs, the similarity is rather high, and for the dissimilar songs, the similarity is overall high except for two pairs of songs.
#This model as-is is not the best representation of similarity between songs but it's not the worst either.